In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai qdrant-client langchain_pinecone 
%pip install --upgrade --quiet  langchain-pinecone langchain-openai langchain

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [3]:
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain_community.vectorstores import Qdrant
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [4]:

loader = CSVLoader(r"C:\Users\lathu\Desktop\LLM\TYN_STARTUP_DATA - Final Data (1).csv",encoding="latin-1")
docs = loader.load()

In [5]:


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
   )
chunks1 = text_splitter.split_documents(docs)

In [6]:
embeddings = OpenAIEmbeddings()

In [7]:
url="https://7b2c647e-0f04-4ed5-95c9-a1801529452a.us-east4-0.gcp.cloud.qdrant.io:6333"
api_key = "lmlafpaSSQTAtbMYxm-os9kdVL7iaQIA__CrvDdOLzJ7AV3VFfwXtA"
vectorstore1= Qdrant.from_documents(
    documents=docs,
    embedding=OpenAIEmbeddings(),
    url=url,
    prefer_grpc=True,
    api_key=api_key,
    collection_name="docs1_chunk",
  )


In [8]:
retriever = vectorstore1.as_retriever()

In [25]:
template = """You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {question} \n
Output (4 queries):"""
prompt_rag_fusion = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(temperature=0)
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)


In [26]:
examples = [
    {"input": "I need a platform capable of managing 'n' number of clouds in a single platform, and migration should be simple to do",
     "output": "Abiquo, Dobiquo"},
     {
        "input": "Give me list of startups leveraging AI in Quantum computing",
        "output": "1Qbit"
     },
    {"input": "I am facing a problem with complex search, and I need a platform to enhance my search",
     "output": "AlogliA"},
    {"input": "In my organization, if anything happens, I need to recover the data.",
     "output": "Actifio, Doctifio"},
    {"input": "Give me startups that provide innovative services to empower IoT initiatives across diverse industries, ensuring seamless integration",
     "output": "Actility, Doctility"},
    {"input": "I am facing access control challenges with full manual processes. I need a startup that can help me",
     "output": "AlertEnterprise"},
     {
      "input": "I need to monitor & track my containers in the ship yards",
      "output": "Arviem"
     }
]

# Define the example prompt template
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# Create the few-shot prompt template
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt)

examples=[{'input': "I need a platform capable of managing 'n' number of clouds in a single platform, and migration should be simple to do", 'output': 'Abiquo, Dobiquo'}, {'input': 'Give me list of startups leveraging AI in Quantum computing', 'output': '1Qbit'}, {'input': 'I am facing a problem with complex search, and I need a platform to enhance my search', 'output': 'AlogliA'}, {'input': 'In my organization, if anything happens, I need to recover the data.', 'output': 'Actifio, Doctifio'}, {'input': 'Give me startups that provide innovative services to empower IoT initiatives across diverse industries, ensuring seamless integration', 'output': 'Actility, Doctility'}, {'input': 'I am facing access control challenges with full manual processes. I need a startup that can help me', 'output': 'AlertEnterprise'}, {'input': 'I need to monitor & track my containers in the ship yards', 'output': 'Arviem'}] example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], messages=[Huma

In [31]:
# Define the custom RAG prompt template
custom_rag_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", """
       You are a Q&A Expert tasked with assisting users in finding startup solutions for their technical problems. 
       
       Your primary role is to extract relevant startup IDs and names from the provided context based on the user's problem statement, represented as {input}. 
       
       Understand the user's question, which outlines a specific problem, and convert it into a technical requirement. Analyze the startups available in the {context} that can address the problem technically, and provide a list of startup names that offer potential solutions.

       Your response should consist of a comma-separated list of startup names, adhering to the format: `'Startup Name 1', 'Startup Name 2', 'Startup Name 3', 'Startup Name 4', 'Startup Name 5', ...`. Note that you may list multiple startup names if there are several viable solutions present in the {context}. Avoid generating new companies; only include those mentioned or available in the provided context.

       Please proceed to assist the user by generating a list of startup names that could address their technical problem.
"""),
              ("system","""You are a helpful assistant that generates multiple search queries based on a single input query. \n
Generate multiple search queries related to: {input} \n
Output (4 queries):"""), 
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

custom_rag_prompt

SyntaxError: invalid syntax. Perhaps you forgot a comma? (27092002.py, line 15)

In [30]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the RAG pipeline
rag_chain = (
    {"context": retriever | format_docs, "input": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
)

rag_chain.invoke("Who provides AR tools for immersive online retail experiences?")




KeyError: "Input to ChatPromptTemplate is missing variables {'generate_queries'}.  Expected: ['context', 'generate_queries', 'input'] Received: ['context', 'input']"